In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/esoh/"
oCV_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

from scipy.io import savemat


In [2]:
## Added from old vrsion 


In [3]:
parameter_values = get_parameter_values()

In [4]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
# spm.print_parameter_info()
param=spm.param

In [5]:
cell = 1

In [6]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [7]:
# pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")

drive_cycle = pd.read_csv(r"C:\Users\Hamidreza\Desktop\CRC\Drive Cycle\US06_default_5Ah.csv", comment="#", header=None).to_numpy()
drive_cycle2 = pd.read_csv(r"C:\Users\Hamidreza\Desktop\CRC\Drive Cycle\US06_GM_5Ah.csv", comment="#", header=None).to_numpy()

experiment = pybamm.Experiment( # defult D.C. DRC-DRC *1.25
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1.9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment2 = pybamm.Experiment( #lab cycling
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *300,
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)

## GM_DCRDCR
experiment3 = pybamm.Experiment( #GM_ DRC-DRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment4 = pybamm.Experiment( #Gm-RCRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment5 = pybamm.Experiment( # defult D.C. RC-RC
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.8 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.7 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

In [8]:
parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Positive electrode LAM constant proportional term [s-1]": 2.5112e-07,
        "Negative electrode LAM constant proportional term [s-1]": 2.4741e-06,
        "Positive electrode LAM constant exponential term": 1.1942,
        "Negative electrode LAM constant exponential term": 1.1942,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
        "SEI kinetic rate constant [m.s-1]": 1.08494281e-16,
        "EC diffusivity [m2.s-1]": 8.30909086e-19,
        "SEI growth activation energy [J.mol-1]": 1.58777981e+04,
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        # "EC diffusivity [m2.s-1]": 2e-18,
        # "SEI growth activation energy [J.mol-1]": 0,
    },
    check_already_exists=False,
)


In [9]:
# all_sumvars_dict1 = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)

In [10]:
sim_long = pybamm.Simulation(spm, experiment=experiment5, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=SOC_0, save_at_cycles=10  )

2022-10-23 15:57:31.857 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/540 (20.284 ms elapsed) --------------------
2022-10-23 15:57:31.857 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 1/14: Run US06 (A) for 1 hours
2022-10-23 15:57:33.010 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 2/14: Rest for 5 minutes
2022-10-23 15:57:33.088 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 3/14: Rest for 1 hours
2022-10-23 15:57:33.137 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 4/14: Rest for 5 minutes
2022-10-23 15:57:33.143 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 15:57:33.293 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 6/14: Hold at 4.2V for 1.8 hours
2022-10-23 15:57:33.687 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 7/14: Rest for 20 minutes
2022-10-23 15:57:33.707 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 8/14: Run US06 (A) for 

2022-10-23 15:57:49.691 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 9/14: Rest for 5 minutes
2022-10-23 15:57:49.702 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 10/14: Rest for 1 hours
2022-10-23 15:57:49.740 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 11/14: Rest for 5 minutes
2022-10-23 15:57:49.747 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 15:57:49.836 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 13/14: Hold at 4.2V for 3.7 hours
2022-10-23 15:57:50.288 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 14/14: Rest for 20 minutes
2022-10-23 15:57:51.317 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.843 Ah (originally 4.860 Ah, will stop at 2.430 Ah)
2022-10-23 15:57:51.318 - [NOTICE] callbacks.on_cycle_start(174): Cycle 6/540 (19.480 s elapsed) --------------------
2022-10-23 15:57:51.319 - [NOTICE] callbacks.on_step_start(182): Cycle 6/540, step

2022-10-23 15:58:07.090 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 2/14: Rest for 5 minutes
2022-10-23 15:58:07.100 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 3/14: Rest for 1 hours
2022-10-23 15:58:07.150 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 4/14: Rest for 5 minutes
2022-10-23 15:58:07.157 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 15:58:07.213 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 6/14: Hold at 4.2V for 1.8 hours
2022-10-23 15:58:07.478 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 7/14: Rest for 20 minutes
2022-10-23 15:58:07.497 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 8/14: Run US06 (A) for 1 hours
2022-10-23 15:58:08.340 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 9/14: Rest for 5 minutes
2022-10-23 15:58:08.352 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 10/14: Rest for 1 hours
2

2022-10-23 15:58:23.210 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 11/14: Rest for 5 minutes
2022-10-23 15:58:23.220 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 15:58:23.311 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 13/14: Hold at 4.2V for 3.7 hours
2022-10-23 15:58:23.810 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 14/14: Rest for 20 minutes
2022-10-23 15:58:24.822 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.804 Ah (originally 4.860 Ah, will stop at 2.430 Ah)
2022-10-23 15:58:24.823 - [NOTICE] callbacks.on_cycle_start(174): Cycle 15/540 (52.985 s elapsed) --------------------
2022-10-23 15:58:24.824 - [NOTICE] callbacks.on_step_start(182): Cycle 15/540, step 1/14: Run US06 (A) for 1 hours
2022-10-23 15:58:25.655 - [NOTICE] callbacks.on_step_start(182): Cycle 15/540, step 2/14: Rest for 5 minutes
2022-10-23 15:58:25.664 - [NOTICE] callbacks.on_step_start(182): Cyc

2022-10-23 15:58:40.635 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 3/14: Rest for 1 hours
2022-10-23 15:58:40.684 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 4/14: Rest for 5 minutes
2022-10-23 15:58:40.692 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 15:58:40.757 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 6/14: Hold at 4.2V for 1.8 hours
2022-10-23 15:58:40.994 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 7/14: Rest for 20 minutes
2022-10-23 15:58:41.014 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 8/14: Run US06 (A) for 1 hours
2022-10-23 15:58:41.917 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 9/14: Rest for 5 minutes
2022-10-23 15:58:41.931 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 10/14: Rest for 1 hours
2022-10-23 15:58:41.978 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 11/14: Rest for 5 minutes


...

2022-10-23 16:25:56.970 - [NOTICE] callbacks.on_cycle_start(174): Cycle 505/540 (28 minutes, 25 seconds elapsed) --------------------
2022-10-23 16:25:56.970 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 1/14: Run US06 (A) for 1 hours
2022-10-23 16:25:57.760 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 2/14: Rest for 5 minutes
2022-10-23 16:25:57.770 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 3/14: Rest for 1 hours
2022-10-23 16:25:57.810 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 4/14: Rest for 5 minutes
2022-10-23 16:25:57.820 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 16:25:57.870 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 6/14: Hold at 4.2V for 1.8 hours
2022-10-23 16:25:58.070 - [NOTICE] callbacks.on_step_start(182): Cycle 505/540, step 7/14: Rest for 20 minutes
2022-10-23 16:25:58.090 - [NOTICE] callbacks.on_step_start(182): Cycle 505/54

2022-10-23 16:26:11.332 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 7/14: Rest for 20 minutes
2022-10-23 16:26:11.352 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 8/14: Run US06 (A) for 1 hours
2022-10-23 16:26:12.132 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 9/14: Rest for 5 minutes
2022-10-23 16:26:12.142 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 10/14: Rest for 1 hours
2022-10-23 16:26:12.182 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 11/14: Rest for 5 minutes
2022-10-23 16:26:12.192 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 16:26:12.262 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 13/14: Hold at 4.2V for 3.7 hours
2022-10-23 16:26:12.672 - [NOTICE] callbacks.on_step_start(182): Cycle 509/540, step 14/14: Rest for 20 minutes
2022-10-23 16:26:13.572 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.500 Ah (origi

2022-10-23 16:26:26.033 - [NOTICE] callbacks.on_step_start(182): Cycle 513/540, step 14/14: Rest for 20 minutes
2022-10-23 16:26:26.933 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.481 Ah (originally 4.860 Ah, will stop at 2.430 Ah)
2022-10-23 16:26:26.933 - [NOTICE] callbacks.on_cycle_start(174): Cycle 514/540 (28 minutes, 55 seconds elapsed) --------------------
2022-10-23 16:26:26.933 - [NOTICE] callbacks.on_step_start(182): Cycle 514/540, step 1/14: Run US06 (A) for 1 hours
2022-10-23 16:26:27.703 - [NOTICE] callbacks.on_step_start(182): Cycle 514/540, step 2/14: Rest for 5 minutes
2022-10-23 16:26:27.703 - [NOTICE] callbacks.on_step_start(182): Cycle 514/540, step 3/14: Rest for 1 hours
2022-10-23 16:26:27.743 - [NOTICE] callbacks.on_step_start(182): Cycle 514/540, step 4/14: Rest for 5 minutes
2022-10-23 16:26:27.753 - [NOTICE] callbacks.on_step_start(182): Cycle 514/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 16:26:27.803 - [NOTICE] callbacks.on_step_start(1

2022-10-23 16:26:41.094 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 16:26:41.144 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 6/14: Hold at 4.2V for 1.8 hours
2022-10-23 16:26:41.344 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 7/14: Rest for 20 minutes
2022-10-23 16:26:41.364 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 8/14: Run US06 (A) for 1 hours
2022-10-23 16:26:42.144 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 9/14: Rest for 5 minutes
2022-10-23 16:26:42.154 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 10/14: Rest for 1 hours
2022-10-23 16:26:42.194 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 11/14: Rest for 5 minutes
2022-10-23 16:26:42.204 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 16:26:42.274 - [NOTICE] callbacks.on_step_start(182): Cycle 518/540, step 13/14: 

2022-10-23 16:26:55.565 - [NOTICE] callbacks.on_step_start(182): Cycle 522/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 16:26:55.645 - [NOTICE] callbacks.on_step_start(182): Cycle 522/540, step 13/14: Hold at 4.2V for 3.7 hours
2022-10-23 16:26:56.045 - [NOTICE] callbacks.on_step_start(182): Cycle 522/540, step 14/14: Rest for 20 minutes
2022-10-23 16:26:56.956 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.438 Ah (originally 4.860 Ah, will stop at 2.430 Ah)
2022-10-23 16:26:56.956 - [NOTICE] callbacks.on_cycle_start(174): Cycle 523/540 (29 minutes, 25 seconds elapsed) --------------------
2022-10-23 16:26:56.956 - [NOTICE] callbacks.on_step_start(182): Cycle 523/540, step 1/14: Run US06 (A) for 1 hours
2022-10-23 16:26:57.766 - [NOTICE] callbacks.on_step_start(182): Cycle 523/540, step 2/14: Rest for 5 minutes
2022-10-23 16:26:57.776 - [NOTICE] callbacks.on_step_start(182): Cycle 523/540, step 3/14: Rest for 1 hours
2022-10-23 16:26:57.806 - [NOTICE] callbacks.on_st

In [11]:
qwrqwf

NameError: name 'qwrqwf' is not defined

In [ ]:
pybamm.plot_summary_variables(sol_long)

In [ ]:
sim_long.plot(
    [
#         "Negative particle surface concentration [mol.m-3]",
#         "Electrolyte concentration [mol.m-3]",
#         "Positive particle surface concentration [mol.m-3]",
        "Current [A]",
#         "Negative electrode potential [V]",
#         "Electrolyte potential [V]",
#         "Positive electrode potential [V]",
        "Terminal voltage [V]",
#         "X-averaged cell temperature",
        "Negative electrode SOC",
        "Positive electrode SOC",
#         "SEI thickness [m]",
        "Loss of capacity to SEI [A.h]",
        "Negative electrode capacity [A.h]",
        "Positive electrode capacity [A.h]",
        "Discharge capacity [A.h]"

        
    ]
)

In [ ]:
print(list(sol_long.summary_variables.keys()))

In [ ]:
type(sol_long.summary_variables[list(sol_long.summary_variables.keys())[1]])

In [ ]:
# mdic={}
# for i in range (1):
# # range(len(sol_long.summary_variables)):
# #     stringing=''.join([list(sol_long.summary_variables.keys())[i]])
#     mdic={f"{i}": sol_long.summary_variables[list(sol_long.summary_variables.keys())[i]]}
# savemat("summary",mdic)


In [14]:
ident=""

In [15]:
x100=sol_long.summary_variables["x_100"]
y100=sol_long.summary_variables["y_100"]
x0=sol_long.summary_variables["x_0"]
y0=sol_long.summary_variables["y_0"]
Cap=sol_long.summary_variables["Capacity [A.h]"]
LoC_SEI_summ=sol_long.summary_variables["Loss of capacity to SEI [A.h]"]
LAM_P=sol_long.summary_variables["Loss of active material in positive electrode [%]"]
LAM_N=sol_long.summary_variables["Loss of active material in negative electrode [%]"]
C_Neg=sol_long.summary_variables["C_n"]
C_pos=sol_long.summary_variables["C_p"]
LLI=sol_long.summary_variables["Loss of lithium inventory [%]"]
cycle_num=sol_long.summary_variables["Cycle number"]
t = sol_long["Time [s]"].entries
I = sol_long["Current [A]"].entries
Q = sol_long['Discharge capacity [A.h]'].entries
LoC_SEI = sol_long["Loss of capacity to SEI [A.h]"].entries
N_SoC = sol_long["Negative electrode SOC"].entries
P_SoC = sol_long["Positive electrode SOC"].entries


ECM_whole_res=sol_long["Local ECM resistance [Ohm]"].entries
ECM_sum_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]
ECM_sum_res_change=sol_long.summary_variables['Change in local ECM resistance [Ohm]']
cycle_time=sol_long.summary_variables['Time [s]']
# exp = 30e6*sol_0["Cell thickness change [m]"].entries
# exp_n = 30e6*sol_0["Negative Electrode thickness change [m]"].entries
# exp_p = 30e6*sol_0["Positive Electrode thickness change [m]"].entries
# ee_p = sol_0["X-averaged positive electrode porosity"].entries
# ee_n = sol_0["X-averaged negative electrode porosity"].entries
# ee_s = sol_0["X-averaged separator porosity"].entries
# j_n = sol_0["X-averaged negative electrode total interfacial current density [A.m-2]"].entries
# j_p = sol_0["X-averaged positive electrode total interfacial current density [A.m-2]"].entries

# x = sol_0["x [m]"].entries
# c_e_0 = sol_0["Electrolyte concentration [mol.m-3]"].entries

In [16]:
mdic = {"x100"+ident: x100, "y100"+ident:y100, "x0"+ident:x0,"y0"+ident:y0,"Cap"+ident:Cap,
        "LoC_SEI_summ"+ident:LoC_SEI_summ,
        "LAM_P"+ident:LAM_P,"LAM_N"+ident:LAM_N,"C_Neg"+ident:C_Neg,"C_pos"+ident:C_pos,
        "LLI"+ident:LLI,"cycle_num"+ident:cycle_num,
        "t"+ident:t,"LoC_SEI"+ident:LoC_SEI,"N_SoC"+ident:N_SoC,"P_SoC"+ident:P_SoC,
        "Q"+ident:Q, "I"+ident:I,  
        "ECM_whole_res":ECM_whole_res,"ECM_sum_res":ECM_sum_res,
        "ECM_sum_res_change":ECM_sum_res_change,"cycle_time":cycle_time         
       }
savemat("Outputs_Oct23_default_RC_RC",mdic)

In [ ]:
# savemat("x100_test", mdic)


In [ ]:
# print(x100)

In [ ]:
# import csv
# with open(r"F:\Git Hub rep\CRC\neew\PyBaMM\Untitled Folder\whatever.csv", 'w') as f:
#     # create the csv writer
#     writer = csv.writer(f)

# write a row to the csv file
# i=1
# for lines in sol_long.all_summary_variables:
# #     writer.writerow(words)
#     i=i+1
#     savemat(f'{i:02d}', lines)


In [ ]:
type(x100)

In [ ]:
sol_long.variables.search("charge")

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Current [A]"].entries[1:15133])

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Terminal voltage [V]"].entries[1:15133])

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [ ]:
sol_long["Time [s]"].entries

In [ ]:
spm.variables.search("resistance")

In [12]:
sim_long.plot(["Local ECM resistance",
"Local ECM resistance [Ohm]",
               "Resistance [Ohm]"])


interactive(children=(FloatSlider(value=0.0, description='t', max=6686.658426834912, step=66.86658426834911), …

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [13]:
sol_long.summary_variables 

{'Minimum measured discharge capacity [A.h]': array([0.00000000e+00, 8.63434572e-05, 1.72652446e-04, 2.59070486e-04,
        3.45556641e-04, 4.32106720e-04, 5.18791780e-04, 6.05562312e-04,
        6.92402647e-04, 7.79401740e-04, 8.66523632e-04, 9.53738603e-04,
        1.04100412e-03, 1.12833413e-03, 1.21580291e-03, 1.30337913e-03,
        1.39103017e-03, 1.47875033e-03, 1.56656873e-03, 1.65441332e-03,
        1.74242604e-03, 1.83058763e-03, 1.91876901e-03, 2.00702683e-03,
        2.09532464e-03, 2.18376283e-03, 2.27228251e-03, 2.36085289e-03,
        2.44952601e-03, 2.53827925e-03, 2.62708948e-03, 2.71599872e-03,
        2.80492154e-03, 2.89400117e-03, 2.98317088e-03, 3.07239367e-03,
        3.16170418e-03, 3.25109672e-03, 3.34058825e-03, 3.43014785e-03,
        3.51977167e-03, 3.60944473e-03, 3.69922667e-03, 3.78909037e-03,
        3.87902545e-03, 3.96904646e-03, 4.05911471e-03, 4.14923205e-03,
        4.23944896e-03, 4.32974582e-03, 4.42012886e-03, 4.51056965e-03,
        4.60102965e